In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#imports
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/text-normalization-challenge-english-language/en_train.csv.zip')
test1 = pd.read_csv('../input/text-normalization-challenge-english-language/en_test.csv.zip')
test2 = pd.read_csv('../input/text-normalization-challenge-english-language/en_test_2.csv.zip')

In [ ]:
print('test_1 data shape',test2.shape)
test1.head()

In [ ]:
print('test_2 data  shape',test2.shape)
test2.head()

In [ ]:
test_list = [test1,test2]
test = pd.concat(test_list)
print('test data  shape',test.shape)
test.head()

In [ ]:
print('train data shape',train.shape)
train.head()

* Each sentence and each oken has an id
* Each token has a class to show its type
* the main goal of the project is to transorm tokens of the column "before to those of the column "after" 

In [ ]:
(test.shape[0])/(test.shape[0]+train.shape[0])

NB: the test size = 0.17 (roughly equal to 0.2)

***let's take a look at the submission dataset***

In [ ]:
submission = pd.read_csv("../input/text-normalization-challenge-english-language/en_sample_submission.csv.zip")
submission.head()


The submission dataset has an id that contains the sentence and the token id as sentence_token combination. 

 ***the Number of sentences***

In [ ]:
print('the number of sentences in the train dataset: ',len(train.sentence_id.unique()))

In [ ]:
print('the number of sentences in the test dataset: ',len(test.sentence_id.unique()))

***the Number of tokens***

In [ ]:
train_sentences =train.groupby('sentence_id')["sentence_id"].count()
print('the number of tokens in each sentence')
print(train_sentences.head())
print('--------------------------------------------------------------------------')
print(train_sentences.describe())


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,10))
sns.set_style("whitegrid")
count_length_fig = sns.countplot(train_sentences , ax=ax)
for item in count_length_fig.get_xticklabels():
    item.set_rotation(90)
 
plt.title('tokens in each sentenes in the train set')
plt.legend()
plt.show()

In [ ]:
#test
test_sentences =test.groupby('sentence_id')["sentence_id"].count()
print('the number of tokens in each sentence')
print(test_sentences.head())
print('--------------------------------------------------------------------------')
print(test_sentences.describe())

Mean and quantiles are shifted a bit to higher values.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,10))
sns.set_style("whitegrid")
count_length_fig = sns.countplot(train_sentences , ax=ax)
for item in count_length_fig.get_xticklabels():
    item.set_rotation(90)
plt.title('tokens in each sentenes in the test set')  
plt.legend()
plt.show()

We notice a peak for sentences that are 7 tokens long in both train and test set.

In [ ]:
#the longest train sentence
max_id = train_sentences[train_sentences == train_sentences.max()].index.values
long_example = train[train.sentence_id==max_id[0]].before.values.tolist()
long_example = ' '.join(long_example)
print('id={}: {}'.format(max_id,long_example))

Looks like a literature reference.

In [ ]:
#some samllest sentence
min_id = train_sentences[train_sentences == train_sentences.min()].index.values
for n in range(5):
    small_example = train[train.sentence_id==min_id[n]].before.values.tolist()
    small_example= ' '.join(small_example)
    print('id={}: {}'.format(min_id[n],small_example))



Dates

In [ ]:
# some of the median sentences
median_id = train_sentences[train_sentences == train_sentences.median()].index.values
for n in range(5):
    median_example = train[train.sentence_id==median_id[n]].before.values.tolist()
    median_example= ' '.join(median_example)
    print('id={}: {}'.format(median_id[n],median_example))


# ***Tokens***

In [ ]:
len(train.token_id.unique())

after reading the explanations for the data in detail: Each token within a sentence has a token_id. Consequently the longest sentence has token ids ranging from 0 to 255 (256), and one of the smallest from 0 to 1.

In [ ]:
print('we have {} token class. '.format(len(train['class'].unique())))

In [ ]:
#train['class'].value_counts()
train.groupby("class")["class"].count()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,12))
#sns.set_style("whitegrid")
count_classes_fig = sns.countplot(y="class", data=train, ax=ax)
for item in count_classes_fig.get_xticklabels():
    item.set_rotation(45)


Most of classes are plain. But there are also some exotic classes.

Understanding of the **ELECTRONIC** and **VERBATIM** classes.

**ELECTRONIC**

In [ ]:
most_electronic_cases = train[train["class"]=='ELECTRONIC'].groupby("before")["before"].count(
).sort_values(ascending=False).head()
sns.barplot(most_electronic_cases.index, most_electronic_cases.values)

**VERBATIM**

In [ ]:
most_verbatim_cases = train[train["class"]=='VERBATIM'].groupby("before")["before"].count(
).sort_values(ascending=False).head(15)

sns.barplot(most_verbatim_cases.index, most_verbatim_cases.values)
plt.show()

**Most of Before words**

In [ ]:
print('unique before words in train: ',len(train.before.unique()))
print('unique before words in test: ',len(test.before.unique()))

In [ ]:
#train train
train_word_counts = train.groupby("before")["before"].count().sort_values(ascending=False).head(15)

sns.barplot(x=train_word_counts.index, y=train_word_counts.values)
plt.show()

In [ ]:
#test set
test_word_counts = test.groupby("before")["before"].count().sort_values(ascending=False).head(15)

sns.barplot(x=test_word_counts.index, y=test_word_counts.values)
plt.show()

# **Pattern of changes**

The before tokens changed after normalization in the train set.

In [ ]:
train["change"] = 0
train.loc[train.before!=train.after, "change"] = 1
train["change"].value_counts()

The tokens changed most often.

In [ ]:
most_changed_words = train[train.change==1].groupby("before")["before"].count(
).sort_values(ascending=False).head(20)

sns.barplot(x=most_changed_words.index, y=most_changed_words.values)
plt.show()

**To which class do the changed words belong most often?**

In [ ]:
changes_classes_fig = sns.countplot(x="class", data=train[train.change==1])
for item in changes_classes_fig.get_xticklabels():
    item.set_rotation(45)

**overlapping class contents**

Actually I find it a bit strange that we have a digit class and a decimal, date, cardinal, ordinal, fraction class etc.. Are the tokens in these classes really all of a different kind or can we find the same token for example in date AND digit? And if so, why are they classified this or that way (does this depend on the sentence content?).

In [ ]:
unique_digits = set(train[train["class"]=="DIGIT"].before.unique().tolist())

unique_dates = set(train[train["class"]=="DATE"].before.unique().tolist())
#we can see other columns like DECIMAl

In [ ]:
overlap = unique_digits.intersection(unique_dates)
len(overlap)

Yes, as we can see by this exampke: We have overlapping class contents!!

This could be good or bad. On the one hand it could depend on the sentence content to which class a token was assigned to. Then the class would also contain this context information implicitly. But on the other hand what if this is not true and the class assignment is somehow dirty or not finetuned enough? Then this opens the door for class feature engineering.